# 模型优化------量化
参考[tensorflow模型优化指南](https://tensorflow.google.cn/model_optimization/guide/quantization/training)
有两种形式的量化：训练后量化和量化感知训练。训练后量化易于使用，但是量化过程中存在一定的精度损失；量化感知训练，对模型量化时考虑了量化带来的精度损失问题，因而量化后模型精度损失表现的更好。
## 量化的数学原理
参考[神经网络量化入门--基本原理](https://zhuanlan.zhihu.com/p/149659607?from_voters_page=true)
量化的目的就是减少网络的大小，以整形运算（或者float16）取代float32运算来加快网络的计算速度。我们知道网络inference过程中绝大部分的运算都是乘加运算。以整形量化为例，介绍量化的数学原理（乘加运算这里做了简化，只包含乘法运算，原理都是相通的）：
$$r = S(q - Z）                                       （1）$$
$$S = \frac{r_{max} - r_{min}}{q_{max} - q_{min}}     （2）$$
$$Z = round(q_{max} - \frac{r_{max}}{S})              （3）$$

*r: 实际值 q:量化值 Z: 量化零点 S: 缩放尺度*

对于乘法运算，我们可以用它的量化值和已知的参数S和Z表示成

$$r_3 = r_1 r_2                                        （4）$$
$$S_3(q_3 - Z_3) = S_1(q_1 - Z_1) S_2(q_2 - Z_2)       （5）$$
最终浮点运算可以用整数运算代替，求得$r_3$的量化值$q_3$,根据(1)式进行反量化可以求得$r_3$，通常网络各层内和层之间的运算我们都是用量化值进行，对于网络的输出执行反量化得到实际输出值。


## 训练后量化
参考[训练后量化](https://tensorflow.google.cn/lite/performance/post_training_quantization)
量化一般是指把模型中的float32运算用float16或者uint8或者int8代替，从而减小模型大小，加快运算速度，减少延时。

**为什么要使用训练后量化？**

我们在训练一个模型的时候，通常用的是浮点类型数据（一般是float32），它可以保证训练的精度。但是使用浮点类型数据的弊端也很明显，模型体积大，运算耗时，CPU和硬件加速器的延迟大。通过量化已经训练好的模型（譬如量化成整形）可以优化以上弊端，付出的代价是可以容忍的模型精度损失。

**如何量化一个已经训练好的模型？**

采用TensorFlow Lite Converter将模型转换为TensorFlow Lite格式，并执行相应的量化操作（这部分不详细讲具体如何实现，感兴趣可以查看官方说明和示例，下文量化感知训练的示例程序中有量化这一部分；我们使用的acuity工具的量化过程原理相同）。

下面我们来简单说一下训练后量化：

在量化的时候需要指定一个数据集（有代表性的数据集）来完成量化过程，实际上量化过程中对于数据集执行inference操作，这样可以记录网络内各层的输出值（每一层的输出都是下一层的输入，最后的输出层除外）的范围，最后执行量化（当然还包括权重和偏移的量化，权重和偏移在训练完之后就是固定的，即使不执行inference我们也可以对他执行量化）。对于uint8整形量化而言，我们查看quantize文件可以看到各层的min_value、max_value、zero_point、scale，它是将实际的float32数值的[mix_value, max_value]空间通过scale和zero_point映射到了uint8整形量化空间[0, 255]。

**量化效果**

据[训练后量化](https://tensorflow.google.cn/lite/performance/post_training_quantization)，整形量化可以将模型缩小4倍，速度提升3倍。


## 量化感知训练
参考[tensorflow模型优化指南](https://tensorflow.google.cn/model_optimization/guide/quantization/training)

**为什么要使用量化感知训练？**

训练后量化会不可避免的把量化过程中的误差引入模型，导致模型精度下降。如果我们在训练模型的时候考虑到这一点那么就可以进一步减小量化带来的精度损失，这就是量化感知训练所要做的。
量化感知训练可以用于已经训练好的网络，也可以用于一个新的待训练的网络(一般推荐使用前者)。

**量化感知训练是如何工作的？**

量化感知训练在inference(forward)时采用了fake quantization，在backword时依然采用浮点值(e.g. float32)。fake quantization对于需要量化的值先做量化(.e.g int8)，紧接着做反量化(.e.g float32)，然后用反量化的浮点值去运算。这样在训练的时候可以将量化引入的误差考虑在内，inference阶段这一误差最终体现在loss中，backword阶段也会去纠正这一误差带来的影响，从而达到减小精度损失的目的。

### 一个量化感知训练的应用示例

**考虑到以下几点：**
+ 大多数情况下我们是在一个已经训练好的网络上执行fine-tune
+ 考虑到我们电脑的算力，可以采用一个较简单的网络
+ 完成量化感知训练后我们还需要部署我们的网络

我们选用lenet网络在fine-tune阶段执行Quantization Aware Training，训练完成后进行部署。

**QAT时应注意：**
+ 与从头开始训练相比，使用量化感知训练进行微调的效果一般更好
+ 尝试“量化某些层”以跳过量化对准确率影响最大的层
+ 尝试量化后面的层而不是前面的层
+ 避免量化关键层

*我在用lenet实验的时候对整个预训练模型进行QAT，测试精度很差，所以只对全连接层进行了QAT*

**如何部署**

官网给的例子是将keras格式的QAT模型转为TFLite格式，然后执行量化。但是这样QAT模型的伪量化还是会被保留下来。

正确的做法应该是把QAT模型各层伪量化阶段量化后的权重取出来，用它来覆盖量化后的模型（这样得对整个模型做QAT吧）。

我这里没有这样做，这样做有点麻烦，我直接把做QAT之后模型的各层权重取出来覆盖预训练模型，得到新模型。然后验证新模型的准确率：把新模型转为TFLite格式，然后量化。实验结果显示这样的方式精度提升了1 pa。可能这样做是不正确的，以后再来改正。

*示例程序参见"quantization_aware_training.ipynb"*